In [1]:
import pandas as pd
import numpy as np
from textblob import TextBlob, Word
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Extracting cleaned dataset
tickets_df = pd.read_csv('tickets_cleaned.csv')

In [3]:
tickets_df.head()

,BMCSERVICEDESK__IMPACT_ID__C,BMCSERVICEDESK__PRIORITY_ID__C,BMCSERVICEDESK__QUEUE__C,BMCSERVICEDESK__ISSERVICEREQUEST__C,FP_CLIENT_NAME__C,FP_DEPARTMENT__C,FP_DESCRIBE_YOUR_ISSUE__C,FP_PARENT_CATEGORY__C,FP_PARENT_TREE__C,CA_FROM_EMAIL_ADDRESS__C,BMCServiceDesk__Type__c,BMCServiceDesk__queueName__c,BMCServiceDesk__shortDescription__c,Parent_Category__c,Category_Action__c,sentiment,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,Single User,5,ServiceDesk,0,FleetPride User,fleetpride.user@fleetpride.com,The terms report was empty today.,Administration,Administration-->IT Services,integrations@fleetpride.com,1,ServiceDesk,Workday Report - Terms Empty,Administration,Delete,-0.1,NaN,NaN,NaN,NaN
1,Single User,5,ServiceDesk,1,FleetPride User,fleetpride.user@fleetpride.com,The change report was empty today.,Administration,Administration-->IT Services,integrations@fleetpride.com,0,ServiceDesk,Workday Report - Changes Empty,Administration,Modify,-0.1,NaN,NaN,NaN,NaN
2,Single User,5,ServiceDesk,1,FleetPride User,fleetpride.user@fleetpride.com,Please double click on the attachment and then...,Administration,Administration-->IT Services,integrations@fleetpride.com,0,ServiceDesk,Workday Report - New Hires,Administration,Delete,0.1214285714,NaN,NaN,NaN,NaN
3,Department/Multi-User,4,Business Analysis,0,FleetPride User,fleetpride.user@fleetpride.com,**ALERT** 882698/QTMHHTTP/VALENCE41P PROD: CPF...,Development Tools and Technologies,Development Tools and Technologies-->IT Services,nightadm@fleetpride.com,1,Business Analysis,Alert/**ALERT** 882698/QTMHHTTP/VALENCE41P,Development Tools and Technologies,Access,0,NaN,NaN,NaN,NaN
4,Department/Multi-User,4,Business Analysis,0,FleetPride User,fleetpride.user@fleetpride.com,**ALERT** 882705/QTMHHTTP/VALENCE41P PROD: CPF...,Development Tools and Technologies,Development Tools and Technologies-->IT Services,nightadm@fleetpride.com,1,Business Analysis,Alert/**ALERT** 882705/QTMHHTTP/VALENCE41P,Development Tools and Technologies,Access,0,NaN,NaN,NaN,NaN


In [4]:
# Only considering relevant features
tickets_df = tickets_df[['BMCSERVICEDESK__IMPACT_ID__C','BMCSERVICEDESK__PRIORITY_ID__C','BMCSERVICEDESK__QUEUE__C','BMCSERVICEDESK__ISSERVICEREQUEST__C','FP_CLIENT_NAME__C','FP_DEPARTMENT__C','FP_DESCRIBE_YOUR_ISSUE__C','FP_PARENT_CATEGORY__C','FP_PARENT_TREE__C','CA_FROM_EMAIL_ADDRESS__C','BMCServiceDesk__Type__c','BMCServiceDesk__queueName__c','BMCServiceDesk__shortDescription__c','Parent_Category__c','Category_Action__c']]

In [5]:
tickets_df

,BMCSERVICEDESK__IMPACT_ID__C,BMCSERVICEDESK__PRIORITY_ID__C,BMCSERVICEDESK__QUEUE__C,BMCSERVICEDESK__ISSERVICEREQUEST__C,FP_CLIENT_NAME__C,FP_DEPARTMENT__C,FP_DESCRIBE_YOUR_ISSUE__C,FP_PARENT_CATEGORY__C,FP_PARENT_TREE__C,CA_FROM_EMAIL_ADDRESS__C,BMCServiceDesk__Type__c,BMCServiceDesk__queueName__c,BMCServiceDesk__shortDescription__c,Parent_Category__c,Category_Action__c
0,Single User,5,ServiceDesk,0,FleetPride User,fleetpride.user@fleetpride.com,The terms report was empty today.,Administration,Administration-->IT Services,integrations@fleetpride.com,1,ServiceDesk,Workday Report - Terms Empty,Administration,Delete
1,Single User,5,ServiceDesk,1,FleetPride User,fleetpride.user@fleetpride.com,The change report was empty today.,Administration,Administration-->IT Services,integrations@fleetpride.com,0,ServiceDesk,Workday Report - Changes Empty,Administration,Modify
2,Single User,5,ServiceDesk,1,FleetPride User,fleetpride.user@fleetpride.com,Please double click on the attachment and then...,Administration,Administration-->IT Services,integrations@fleetpride.com,0,ServiceDesk,Workday Report - New Hires,Administration,Delete
3,Department/Multi-User,4,Business Analysis,0,FleetPride User,fleetpride.user@fleetpride.com,**ALERT** 882698/QTMHHTTP/VALENCE41P PROD: CPF...,Development Tools and Technologies,Development Tools and Technologies-->IT Services,nightadm@fleetpride.com,1,Business Analysis,Alert/**ALERT** 882698/QTMHHTTP/VALENCE41P,Development Tools and Technologies,Access
4,Department/Multi-User,4,Business Analysis,0,FleetPride User,fleetpride.user@fleetpride.com,**ALERT** 882705/QTMHHTTP/VALENCE41P PROD: CPF...,Development Tools and Technologies,Development Tools and Technologies-->IT Services,nightadm@fleetpride.com,1,Business Analysis,Alert/**ALERT** 882705/QTMHHTTP/VALENCE41P,Development Tools and Technologies,Access
5,Department/Multi-User,4,Business Analysis,0,FleetPride User,fleetpride.user@fleetpride.com,**ALERT** 888975/MBUROUGH/MES_SAVE PROD: CPF M...,iSeries Business Applications,iSeries Business Applications-->IT Services,nightadm@fleetpride.com,1,Business Analysis,Alert/**ALERT** 888975/MBUROUGH/MES_SAVE,iSeries Business Applications,Data Update
6,Single User,5,ServiceDesk,0,Wayne Lassiter,Raleigh,"Greetings, The IT Department has been informed...",Administration,Administration-->IT Services,wayne.lassiter@fleetpride.com,1,ServiceDesk,Term Asset Recovery - Brett Yarborough - Cell...,Administration,Delete
7,Department/Multi-User,4,Business Analysis,0,FleetPride User,fleetpride.user@fleetpride.com,**ALERT** 889830/CJONES3/OGD_SAVE PROD: CPF MS...,iSeries Business Applications,iSeries Business Applications-->IT Services,nightadm@fleetpride.com,1,Business Analysis,Alert/**ALERT** 889830/CJONES3/OGD_SAVE,iSeries Business Applications,Data Extract
8,Single User,5,ServiceDesk,0,Kelley Groves,Salt Lake City,"Greetings, The IT Department has been informed...",Administration,Administration-->IT Services,kelley.groves@fleetpride.com,1,ServiceDesk,Term Asset Recovery - James Tucker,Administration,Delete
9,Department/Multi-User,4,Business Analysis,0,FleetPride User,fleetpride.user@fleetpride.com,**ALERT** 893788/EDIUSER/COCACOLA_I PROD: CPF ...,Data Integration,Data Integration-->IT Services,nightadm@fleetpride.com,1,Business Analysis,Alert/**ALERT** 893788/EDIUSER/COCACOLA_I,Data Integration,Data Extract


In [6]:
#Function to detect sentiment from text
def detect_sentiment(text):
    return TextBlob(text.decode('utf-8')).sentiment.polarity

In [7]:
# Detecting sentiments from problem description
tickets_df['sentiment'] = tickets_df.FP_DESCRIBE_YOUR_ISSUE__C.apply(detect_sentiment)

In [8]:
tickets_df.head()

,BMCSERVICEDESK__IMPACT_ID__C,BMCSERVICEDESK__PRIORITY_ID__C,BMCSERVICEDESK__QUEUE__C,BMCSERVICEDESK__ISSERVICEREQUEST__C,FP_CLIENT_NAME__C,FP_DEPARTMENT__C,FP_DESCRIBE_YOUR_ISSUE__C,FP_PARENT_CATEGORY__C,FP_PARENT_TREE__C,CA_FROM_EMAIL_ADDRESS__C,BMCServiceDesk__Type__c,BMCServiceDesk__queueName__c,BMCServiceDesk__shortDescription__c,Parent_Category__c,Category_Action__c,sentiment
0,Single User,5,ServiceDesk,0,FleetPride User,fleetpride.user@fleetpride.com,The terms report was empty today.,Administration,Administration-->IT Services,integrations@fleetpride.com,1,ServiceDesk,Workday Report - Terms Empty,Administration,Delete,-0.100000
1,Single User,5,ServiceDesk,1,FleetPride User,fleetpride.user@fleetpride.com,The change report was empty today.,Administration,Administration-->IT Services,integrations@fleetpride.com,0,ServiceDesk,Workday Report - Changes Empty,Administration,Modify,-0.100000
2,Single User,5,ServiceDesk,1,FleetPride User,fleetpride.user@fleetpride.com,Please double click on the attachment and then...,Administration,Administration-->IT Services,integrations@fleetpride.com,0,ServiceDesk,Workday Report - New Hires,Administration,Delete,0.121429
3,Department/Multi-User,4,Business Analysis,0,FleetPride User,fleetpride.user@fleetpride.com,**ALERT** 882698/QTMHHTTP/VALENCE41P PROD: CPF...,Development Tools and Technologies,Development Tools and Technologies-->IT Services,nightadm@fleetpride.com,1,Business Analysis,Alert/**ALERT** 882698/QTMHHTTP/VALENCE41P,Development Tools and Technologies,Access,0.000000
4,Department/Multi-User,4,Business Analysis,0,FleetPride User,fleetpride.user@fleetpride.com,**ALERT** 882705/QTMHHTTP/VALENCE41P PROD: CPF...,Development Tools and Technologies,Development Tools and Technologies-->IT Services,nightadm@fleetpride.com,1,Business Analysis,Alert/**ALERT** 882705/QTMHHTTP/VALENCE41P,Development Tools and Technologies,Access,0.000000


In [9]:
#Text processing
#Dividing the problemDescription-problemType sets into training and test sets 
X_train, X_test, y_train, y_test = train_test_split(tickets_df.FP_DESCRIBE_YOUR_ISSUE__C, tickets_df.BMCServiceDesk__Type__c, random_state=1)

In [10]:
# Using CountVectorizer to create document-term matrices from X_train and X_test
vect = CountVectorizer()
train_dtm = vect.fit_transform(X_train)
test_dtm = vect.transform(X_test)

In [11]:
# rows are documents, columns are terms (features)
train_dtm.shape

(6132, 19208)

In [12]:
# Using Logistic Regression to predict the problem type - (0-NotAnIssue, 1-Issue)
lr = LogisticRegression()
lr.fit(train_dtm, y_train)
y_pred_class = lr.predict(test_dtm)

In [13]:
print metrics.accuracy_score(y_test, y_pred_class) #80% Accuracy

0.8018590998043053


In [14]:
tickets_df.columns.values

array(['BMCSERVICEDESK__IMPACT_ID__C', 'BMCSERVICEDESK__PRIORITY_ID__C',
       'BMCSERVICEDESK__QUEUE__C', 'BMCSERVICEDESK__ISSERVICEREQUEST__C',
       'FP_CLIENT_NAME__C', 'FP_DEPARTMENT__C',
       'FP_DESCRIBE_YOUR_ISSUE__C', 'FP_PARENT_CATEGORY__C',
       'FP_PARENT_TREE__C', 'CA_FROM_EMAIL_ADDRESS__C',
       'BMCServiceDesk__Type__c', 'BMCServiceDesk__queueName__c',
       'BMCServiceDesk__shortDescription__c', 'Parent_Category__c',
       'Category_Action__c', 'sentiment'], dtype=object)

In [15]:
#Using remaining relevant features to make predictions
tickets_df = tickets_df[['BMCSERVICEDESK__IMPACT_ID__C', 'BMCSERVICEDESK__PRIORITY_ID__C',
       'BMCSERVICEDESK__QUEUE__C',
       'FP_CLIENT_NAME__C', 
       'FP_PARENT_TREE__C', 'BMCServiceDesk__queueName__c',
        'Parent_Category__c','Category_Action__c', 'sentiment',
       'BMCServiceDesk__Type__c']]

In [16]:
# One hot encoding to convert categorical features into numerical ones
tickets_df = pd.get_dummies(tickets_df,columns=['BMCSERVICEDESK__IMPACT_ID__C', 
       'BMCSERVICEDESK__QUEUE__C',
       'FP_CLIENT_NAME__C', 
       'FP_PARENT_TREE__C', 'BMCServiceDesk__queueName__c',
        'Parent_Category__c','Category_Action__c'])

In [17]:
tickets_df.head()

,BMCSERVICEDESK__PRIORITY_ID__C,sentiment,BMCServiceDesk__Type__c,BMCSERVICEDESK__IMPACT_ID__C_Company Wide,BMCSERVICEDESK__IMPACT_ID__C_Department/Multi-User,BMCSERVICEDESK__IMPACT_ID__C_Single User,BMCSERVICEDESK__QUEUE__C_Branch Ops,BMCSERVICEDESK__QUEUE__C_Business Analysis,BMCSERVICEDESK__QUEUE__C_Business Intelligence,BMCSERVICEDESK__QUEUE__C_Data Integrity,...,Category_Action__c_Research Question,Category_Action__c_SOM punchout,Category_Action__c_ServiceDesk,Category_Action__c_Spam Blacklist,Category_Action__c_Spam Whitelist,Category_Action__c_Technical Applications,Category_Action__c_Training Issue,Category_Action__c_Voice Services,Category_Action__c_iSeries Business Applications,Category_Action__c_not a pricing ticket>>>>>>>Scan from a Xerox Phaser MFP
0,5,-0.100000,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5,-0.100000,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5,0.121429,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0.000000,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0.000000,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
tickets_df.shape

(8176, 1224)

In [19]:
X=tickets_df.loc[:,tickets_df.columns!='BMCServiceDesk__Type__c']

In [20]:
y=tickets_df.BMCServiceDesk__Type__c

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [22]:
# Using Logistic Regression to predict the problem type - (0-NotAnIssue, 1-Issue)
lr.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [23]:
y_predictions=lr.predict(X_test)

In [24]:
print metrics.accuracy_score(y_test, y_predictions) # 81% Accuracy

0.8140900195694716
